In [5]:
import json
import os

from constants import *

test_json = [json.loads(a) for a in open(os.path.join(ROOT_PATH, "data", "WikiMed.json"), "r")]

In [8]:
print(test_json[0]["text"])

Polypterus

Polypterus is a genus of freshwater fish in the bichir family (Polypteridae) of order Polypteriformes. The type species is the Nile bichir ("P. bichir"). Fishes in this genus live in various areas in Africa. "Polypterus" is the only known vertebrate to have lungs, but no trachea.

The etymology of the genus name derives from a combination of the Greek prefix πολυ-, "poly-" (many) and the root word πτερον, "pteron" (wing or fin) – "many fins".

In shallow water, "Polypterus" inhales primarily through its spiracle (blowhole). Exhalation is powered by muscles in the torso. During exhalation, the bony scales in the upper chest become indented. When the muscles are relaxed, the bony scales spring back into position, generating negative pressure within the torso, resulting in a rapid intake of air through the spiracle. The air is nearly sufficient to fill the lungs. This is followed by one cycle of buccal (mouth) pumping, which "tops off" the lungs, with the surplus air from the 

In [69]:
import os
import json
import pandas as pd

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

from constants import *

with open(os.path.join(ROOT_PATH, "train_sentences_full.json"), "r") as f:
    list_sentences = json.load(f)

with open(os.path.join(ROOT_PATH, "train_label_id_full.json"), "r") as f:
    list_labels = json.load(f)

data_dict = {"Sentence": list_sentences, "Label": list_labels}
data_df = pd.DataFrame(data_dict)

In [99]:
NUM_TRAIN_PER_CLASS = 40
NUM_TRAIN_NO_RELATION = 2000
NUM_TRAIN_NO_LABEL = 4000

NUM_TEST_PER_CLASS = 20
NUM_TEST_NO_RELATION = 100

NO_LABEL_LABEL = None

train_sentences_full = []
train_labels_full = []
test_sentences_full = []
test_labels_full = []

for i in data_df["Label"].unique():
    
    potential_df = data_df.loc[data_df["Label"] == i]["Sentence"]
    
    if i == 0:
        train_sent, test_sent = train_test_split(potential_df.values, train_size=NUM_TRAIN_NO_RELATION, test_size=NUM_TEST_NO_RELATION)
        train_labels = [int(i)] * NUM_TRAIN_NO_RELATION
        test_labels = [int(i)] * NUM_TEST_NO_RELATION
    else:
        train_sent, test_sent = train_test_split(potential_df.values, train_size=NUM_TRAIN_PER_CLASS, test_size=NUM_TEST_PER_CLASS)
        train_labels = [int(i)] * NUM_TRAIN_PER_CLASS
        test_labels = [int(i)] * NUM_TEST_PER_CLASS
    
    train_sentences_full += list(train_sent)
    test_sentences_full += list(test_sent)
    train_labels_full += train_labels
    test_labels_full += test_labels

# train_sentences_full += random.sample(list(data_df["Sentence"].values), NUM_TRAIN_NO_LABEL)
# train_labels_full += [NO_LABEL_LABEL] * NUM_TRAIN_NO_LABEL

json_o_train_sentences = json.dumps(train_sentences_full)
json_o_test_sentences = json.dumps(test_sentences_full)
json_o_train_labels = json.dumps(train_labels_full)
json_o_test_labels = json.dumps(test_labels_full)
    
with open("train_sentence.json", "w") as f:
    f.write(json_o_train_sentences)

with open("train_label_id.json", "w") as f:
    f.write(json_o_train_labels)

with open("test_sentence.json", "w") as f:
    f.write(json_o_test_sentences)

with open("test_label_id.json", "w") as f:
    f.write(json_o_test_labels)


LE TROU

















Oui

In [1]:
import os
import re
from constants import *

import spacy
import scispacy
import pandas as pd

from tqdm.auto import tqdm

from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector

from termcolor import colored

In [27]:
filename = "20683753_RE.csv"

with open(os.path.join(DATA_CLEAN_PATH, filename.split("_")[0]+".txt"), "r") as f:
    text = f.read()

with open(os.path.join(ENTITIES_PATH, filename.split("_")[0]+".csv"), "r") as f:
    entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)
    
with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
    relations_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

In [52]:
sentences = []

cursor = 0
cnt_sentences = 0
last_end = 0

for word, start_char, end_char in list(entities_df[["Word", "StartChar", "EndChar"]].itertuples(index=False, name=None)):
    dot = re.search("\.", text[cursor:start_char])
    if dot:
        sentences.append({"Text": text[last_end:cursor + dot.span()[0]],
                          "StartChar": last_end,
                          "EndChar": cursor + dot.span()[0]})
        last_end = cursor + dot.span()[1]
        cnt_sentences += 1
    else:
        pass
    cursor = end_char
sentences_df = pd.DataFrame(sentences)

In [78]:
sentences_full_list = []

for first_id, second_id, first_word, second_word in list(relations_df[["First", "End", "FirstWord", "EndWord"]].itertuples(index=False, name=None)):
    sent_id = entities_df.iloc[first_id]["Sentence"]
    sent_text = sentences_df.iloc[sent_id]["Text"]
    sent_start = sentences_df.iloc[sent_id]["StartChar"]
    sent_end = sentences_df.iloc[sent_id]["EndChar"]

    first_start_char = entities_df.iloc[first_id]["StartChar"]
    first_end_char = entities_df.iloc[first_id]["EndChar"]
    second_start_char = entities_df.iloc[second_id]["StartChar"]
    second_end_char = entities_df.iloc[second_id]["EndChar"]
    
    sentence_full = "[CLS] " + sent_text[:first_start_char - sent_start].strip() + \
                    " <e1>" + first_word + "</e1>" + \
                    sent_text[first_end_char - sent_start:second_start_char - sent_start] + \
                    "<e2>" + second_word + "</e2> " + \
                    sent_text[second_end_char - sent_start:].strip() + " [SEP]"    
    print(sentence_full)
    sentences_full_list.append(sentence_full)

[CLS] The present <e1>study</e1> is aiming at elucidating the <e2>effect</e2> of intraoperative lavage with short-chain fatty acids on colonic anastomosis in rats [SEP]
[CLS] The present <e1>study</e1> is aiming at elucidating the effect of <e2>intraoperative lavage</e2> with short-chain fatty acids on colonic anastomosis in rats [SEP]
[CLS] The present <e1>study</e1> is aiming at elucidating the effect of intraoperative lavage with <e2>short-chain fatty acid</e2> on colonic anastomosis in rats [SEP]
[CLS] The present <e1>study</e1> is aiming at elucidating the effect of intraoperative lavage with short-chain fatty acids on <e2>colonic anastomosis</e2> in rats [SEP]
[CLS] The present <e1>study</e1> is aiming at elucidating the effect of intraoperative lavage with short-chain fatty acids on colonic anastomosis in <e2>rat</e2>  [SEP]
[CLS] The present study is aiming at elucidating the <e1>effect</e1> of <e2>intraoperative lavage</e2> with short-chain fatty acids on colonic anastomosis i

[CLS] In the 1st group, no intraoperative large bowel lavage was performed; in the 2nd, a lavage with normal saline solution; in the 3rd, the animals received a diet rich in SCFAs pre- and postoperatively and a lavage with normal saline <e1>0.9%</e1>; and the 4th group received an intraoperative lavage with SCFAs <e2>solution</e2>  [SEP]
[CLS] In the 1st group, no intraoperative large bowel lavage was performed; in the 2nd, a lavage with normal saline solution; in the 3rd, the animals received a diet rich in SCFAs pre- and postoperatively and a lavage with normal saline 0.9%; and the <e1>4th group</e1> received an <e2>intraoperative</e2> lavage with SCFAs solution [SEP]
[CLS] In the 1st group, no intraoperative large bowel lavage was performed; in the 2nd, a lavage with normal saline solution; in the 3rd, the animals received a diet rich in SCFAs pre- and postoperatively and a lavage with normal saline 0.9%; and the <e1>4th group</e1> received an intraoperative lavage with <e2>scfas</e

[CLS] The abdominal wound of the animals of groups 3 and 4 had a better <e1>clinical healing</e1> <e2>appearance</e2> than those of the 2nd group [SEP]
[CLS] The abdominal wound of the animals of groups 3 and 4 had a better <e1>clinical healing</e1> appearance than those of the <e2>2nd group</e2>  [SEP]
[CLS] The abdominal wound of the animals of groups 3 and 4 had a better clinical healing <e1>appearance</e1> than those of the <e2>2nd group</e2>  [SEP]
[CLS] The <e1>animal</e1> of <e2>the 1st, 2nd</e2> and 3rd groups had an increased incidence of anastomotic or abdominal wall abscesses in contrast to those of the 4th group [SEP]
[CLS] The <e1>animal</e1> of the 1st, 2nd and <e2>3rd</e2> groups had an increased incidence of anastomotic or abdominal wall abscesses in contrast to those of the 4th group [SEP]
[CLS] The <e1>animal</e1> of the 1st, 2nd and 3rd <e2>group</e2> had an increased incidence of anastomotic or abdominal wall abscesses in contrast to those of the 4th group [SEP]
[CL

[CLS] By the intraoperative lavage with SCFAs, we provide fuel requirements that may increase <e1>anastomotic strength</e1>, as it was observed in the <e2>4th group</e2> of our experiments [SEP]
[CLS] By the intraoperative lavage with SCFAs, we provide fuel requirements that may increase <e1>anastomotic strength</e1>, as it was observed in the 4th group of our <e2>experiment</e2>  [SEP]
[CLS] By the intraoperative lavage with SCFAs, we provide fuel requirements that may increase anastomotic strength, as it was observed in the <e1>4th group</e1> of our <e2>experiment</e2>  [SEP]
[CLS] On the contrary, <e1>decrease</e1> <e2>strength</e2> was observed in lavaged groups with the saline solution [SEP]
[CLS] On the contrary, <e1>decrease</e1> strength was observed in <e2>lavaged group</e2> with the saline solution [SEP]
[CLS] On the contrary, <e1>decrease</e1> strength was observed in lavaged groups with the <e2>saline solution</e2>  [SEP]
[CLS] On the contrary, decreased <e1>strength</e1> w

IndexError: single positional indexer is out-of-bounds

In [76]:
import json
json_object = json.dumps(sentences_full_list)

with open("train_sentences.json", "w") as f:
    f.write(json_object)

In [11]:
nlp_scispacy = spacy.load("en_core_sci_md")
nlp_scispacy.add_pipe("abbreviation_detector")
nlp_scispacy.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,
                      "linker_name": "umls", "max_entities_per_mention": 1})

ner_models = [
    {"type": "Spacy", "name": "SciSpacy MD", "prio": 0, "model": nlp_scispacy},
    {"type": "Spacy", "name": "Spacy SM", "prio": 1, "model": spacy.load("en_core_web_md")},
]

In [16]:
def get_entities_from_spacy(text, model, name, filename):

    doc = model(text)
    entities = list(doc.ents)
    entities.sort(key=lambda x: x.start_char)

    entities_list = []
    try:
        linker = model.get_pipe("scispacy_linker")
    except:
        linker = None

    for ent in entities:
        word = ent.lemma_
        e_type = ent.label_
        source = name
        start_char = ent.start_char
        end_char = ent.end_char
        document = filename.split('.')[0]
        try:
            CUI, score = ent._.kb_ents[0]
        except:
            CUI, score = None, None
        if CUI and score:
            TUI = re.search(r"T[0-9]{3}", str(linker.kb.cui_to_entity[CUI]))[0]
            group = TUI_MAP[TUI]
        else:
            TUI, group = None, None

        entities_list.append({"Word": word,
                              "Type": e_type,
                              "Source": source,
                              "StartChar": start_char,
                              "EndChar": end_char,
                              "Document": document,
                              "CUI": CUI,
                              "Score": score,
                              "TUI": TUI,
                              "Group": group})

    return entities_list

In [17]:
entities_df = pd.DataFrame()
seen = [0] * len(text)

for model in ner_models:
    model_type = model["type"]
    model_name = model["name"]
    model_prio = model["prio"]
    model_nlp = model["model"]
    if model_type == "Spacy":
        entities_list = get_entities_from_spacy(text, model_nlp, model_name, filename)
    elif model_type == "Manual":
        entities_list = get_entities_from_manual(text, model_nlp, model_name, filename)
    else:
        raise NotImplementedError(f"Model type {model_type} is unknown.")
 
    if len(entities_df) > 0:
        good_entities = []
        for ent in entities_list:
            if sum(seen[ent["StartChar"]:ent["EndChar"]]) == 0:
                good_entities.append(ent)
                seen[ent["StartChar"]:ent["EndChar"]] = [1] * (ent["EndChar"] - ent["StartChar"])
        entities_df = pd.concat([entities_df, pd.DataFrame(good_entities)])
    else:
        for ent in entities_list:
            seen[ent["StartChar"]:ent["EndChar"]] = [1] * (ent["EndChar"] - ent["StartChar"])
        entities_df = pd.DataFrame(entities_list)

if len(entities_df) != 0:
    entities_df = entities_df.sort_values(by=["Document", "StartChar"], axis=0).reset_index(drop=True)

text = re.sub("[^a-zA-Z0-9\.]", " ", text)

cursor = 0
cnt_words = 0
cnt_sentences = 0
words_dicts = []
for word, start_char, end_char in list(entities_df[["Word", "StartChar", "EndChar"]].itertuples(index=False, name=None)):
    cnt_words += sum([len(a) > 0 for a in text[cursor:start_char].split(" ")])
    cnt_sentences += sum([a == "." for a in text[cursor:start_char]])
    start_id = cnt_words
    sentence_id = cnt_sentences
    num_words = sum([len(a) > 0 for a in word.split(" ")])
    end_id = start_id + num_words - 1
    cnt_words += num_words
    words_dicts.append({"Word": word, "StartWord": start_id, "EndWord": end_id, "Sentence": sentence_id})
    cursor = end_char
words_df = pd.DataFrame(words_dicts)

In [20]:
print(text[:2000])

Colorectal anastomotic leakage is a serious complication leading to major postoperative morbidity and mortality. In the present study  we investigated the early detection of anastomotic leakage before its clinical presentation. Fifty six patients with rectal cancer were included prospectively in this study. All patients underwent elective low anterior resection. Peritoneal samples were collected from the abdominal drains at the first  third  and fifth days postoperatively for bacteriological study and cytokines measurement. Patients were divided into two groups  those without symptomatic or clinical evidence of anastomotic leakage and those with clinical evidence of AL. Study variables included hospital stay  wound infection  operative time  blood loss  height of anastomosis  intraperitoneal cytokines  and microbiological study of peritoneal fluid. Clinically evident AL occurred in eight patients and diagnosed postoperatively on median day 6. Intraperitoneal bacterial colonization and 

In [22]:
words_df.head(30)

,Word,StartWord,EndWord,Sentence
0,colorectal,0,0,0
1,anastomotic leakage,1,2,0
2,complication,6,6,0
3,postoperative,10,10,0
4,morbidity,11,11,0
5,mortality,13,13,0
6,study,18,18,1
7,investigate,20,20,1
8,early detection,22,23,1
9,anastomotic leakage,25,26,1


In [3]:
import pandas as pd
with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRDEF")) as f:
    df_types = pd.read_csv(f, delimiter='|', names=["Record Type", "TUI", "Name", "Tree Number", "Definition", "Examples", "Usage note", "non-human flag ?", "Abbreviation", "Inverse"], index_col=False)
    df_types.drop(["Examples", "Usage note", "non-human flag ?"], axis=1, inplace=True)
    sty_df = df_types.loc[df_types["Record Type"] == "STY"].reset_index(drop=True)
    rl_df =  df_types.loc[df_types["Record Type"] == "RL"].reset_index(drop=True)

In [6]:
with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SG")) as f:
    df_group = pd.read_csv(f, delimiter='|', names=["Group Abbrev", "Group Name", "TUI", "Semantic Type"], index_col=False)
TUI_map = {}
for i, infos in df_group[["Group Abbrev", "TUI"]].iterrows():
    TUI_map[infos["TUI"]] = infos["Group Abbrev"]

In [287]:
entities_df

,Word,Type,Source,StartChar,EndChar,Document,CUI,Score,TUI,Group,StartWord,EndWord
0,investigate,ENTITY,SciSpacy MD,3,14,28259203,C1292732,1.0,T169,CONC,1,1
1,effect,ENTITY,SciSpacy MD,19,25,28259203,C1280500,1.0,T080,CONC,3,3
2,blood transfusion,ENTITY,SciSpacy MD,29,47,28259203,C0005841,1.0,T061,PROC,5,6
3,risk,ENTITY,SciSpacy MD,55,59,28259203,C0035647,1.0,T078,CONC,9,9
4,anastomotic leakage,ENTITY,SciSpacy MD,63,82,28259203,C0919691,0.9999998807907104,T046,DISO,11,12
...,...,...,...,...,...,...,...,...,...,...,...,...
941,pathogenesis,ENTITY,SciSpacy MD,18859,18871,28259203,C0543483,1.0,T169,CONC,2780,2780
942,research,ENTITY,SciSpacy MD,18887,18895,28259203,C0035168,1.0,T062,PROC,2783,2783
943,Danish Cancer Society,ENTITY,SciSpacy MD,18949,18970,28259203,UNDEF,UNDEF,UNDEF,UNDEF,2792,2794
944,Aage,ENTITY,SciSpacy MD,18976,18980,28259203,UNDEF,UNDEF,UNDEF,UNDEF,2796,2796


In [288]:
import os
import pandas as pd

from constants import *

MAX_WORD_DIST = 20

relations_dicts = []


def get_UMLS_score(StartTUI, EndTUI, umls_relations_df):
    return len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"]
                   == StartTUI].loc[umls_relations_df["EndTUI"] == EndTUI])

with open(os.path.join(ENTITIES_PATH, filename.split(".")[0]+".csv"), "r") as f:
    entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRSTRE1")) as f:
    umls_relations_df = pd.read_csv(f, delimiter='|', names=["FirstTUI", "RelationTUI", "EndTUI"], index_col=False)

relations_dicts = []

for i in tqdm(range(len(entities_df))):
    forward_df = entities_df.iloc[i + 1:]
    valid_relations = forward_df.loc[forward_df["EndWord"] <= entities_df["StartWord"].iloc[i] + MAX_DIST]
    for j in range(len(valid_relations)):
        relations_dicts.append({"First": i,
                                "End": i + j + 1,
                                "FirstWord": entities_df.iloc[i]["Word"],
                                "EndWord": valid_relations.iloc[j]["Word"],
                                "FirstTUI": entities_df.iloc[i]["TUI"],
                                "EndTUI": valid_relations.iloc[j]["TUI"],
                                "FirstGroup": entities_df.iloc[i]["Group"],
                                "EndGroup": valid_relations.iloc[j]["Group"],
                                "Distance": valid_relations.iloc[j]["StartWord"] - entities_df.iloc[i]["EndWord"],
                                "ScoreUMLS": get_UMLS_score(entities_df.iloc[i]["TUI"], entities_df.iloc[j]["TUI"], umls_relations_df)})

pd.DataFrame(relations_dicts)

  0%|          | 0/946 [00:00<?, ?it/s]

,First,End,FirstWord,EndWord,FirstTUI,EndTUI,FirstGroup,EndGroup,Distance,ScoreUMLS
0,0,1,investigate,effect,T169,T080,CONC,CONC,2,0
1,0,2,investigate,blood transfusion,T169,T061,CONC,PROC,4,0
2,0,3,investigate,risk,T169,T078,CONC,CONC,8,0
3,0,4,investigate,anastomotic leakage,T169,T046,CONC,DISO,10,1
4,0,5,investigate,patient,T169,T101,CONC,LIVB,13,0
...,...,...,...,...,...,...,...,...,...,...
31259,942,944,research,Aage,T062,UNDEF,PROC,UNDEF,13,0
31260,942,945,research,Johanne Louis-Hansens Foundation,T062,UNDEF,PROC,UNDEF,15,0
31261,943,944,Danish Cancer Society,Aage,UNDEF,UNDEF,UNDEF,UNDEF,2,0
31262,943,945,Danish Cancer Society,Johanne Louis-Hansens Foundation,UNDEF,UNDEF,UNDEF,UNDEF,4,0


In [9]:
def get_relations_score(StartTUI, EndTUI, distance, umls_relations_df):
    num_umls = len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"] == StartTUI].loc[umls_relations_df["EndTUI"] == EndTUI])
    if num_umls == 0:
        return STANDARD_SCORE
    else:
        return num_umls * UMLS_SCORE * (1 - (DIST_MULT * (distance - 1) / MAX_DIST))

In [ ]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from constants import *

THRESHOLD = 80

SCORE_UMLS_MULT = 3
STANDARD_SCORE = 1
DIST_CST = 40  # Value used in the exponential, increase to make the distance less impactful

mega_dict = {}

with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRSTRE1")) as f:
    umls_relations_df = pd.read_csv(f, delimiter='|', names=["FirstTUI", "RelationTUI", "EndTUI"], index_col=False)

def get_score(distance, score_umls): # Score UMLS : Number of Matching realtions found in database
    if score_umls == 0:
        return STANDARD_SCORE
    else:
        return score_umls * SCORE_UMLS_MULT * np.exp(-1 * distance / DIST_CST)


for filename in os.listdir(RELATIONS_PATH):
    with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
        relations_df = pd.read_csv(f)

    if "FirstWord" in relations_df.columns and  "EndWord" in relations_df.columns and "Distance" in relations_df.columns and "ScoreUMLS" in relations_df.columns:
        for first_word, end_word, firstTUI, endTUI, distance, score in list(relations_df[["FirstWord", "EndWord", "FirstTUI", "EndTUI", "Distance", "ScoreUMLS"]].itertuples(index=False, name=None)):
            if first_word and first_word not in mega_dict.keys():
                mega_dict[first_word] = {}
            else:
                num_umls = len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"] == firstTUI].loc[umls_relations_df["EndTUI"] == endTUI])
                if end_word and end_word not in mega_dict[first_word].keys():
                    mega_dict[first_word][end_word] = get_score(distance, score)
                else:
                    mega_dict[first_word][end_word] += get_score(distance, score)

In [ ]:
def one_step(k1):
    if k1 not in mega_dict.keys():
        return None
    else:
        print(k1)
        edges = []
        for k2 in mega_dict[k1].keys():
            if mega_dict[k1][k2] >= THRESHOLD:
                print(k2)
                edges += [(k1, k2), one_step(k2)]
        return edges

In [ ]:
K = list(mega_dict.keys())[1]
print(one_step(K))

In [2]:
import os

import spacy
import scispacy
import pandas as pd

from tqdm.auto import tqdm

from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector

from constants import *

In [26]:
nlp_scispacy = spacy.load("en_core_sci_md")
nlp_scispacy.add_pipe("abbreviation_detector")
nlp_scispacy.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,
                      "linker_name": "umls", "max_entities_per_mention": 1})

/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [27]:
filename = "20978948.txt"
with open(os.path.join(DATA_CLEAN_PATH, filename), "r") as f:
    text = f.read()

In [28]:
doc = nlp_scispacy(text)
entities = list(doc.ents)

/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [29]:
for ent in entities:
    print(ent.lemma_)

colorectal
anastomotic leakage
complication
postoperative
morbidity
mortality
study
investigate
early detection
anastomotic leakage
clinical presentation
patient
rectal cancer
study
patient
elective low anterior
resection
peritoneal sample
abdominal drain
day
postoperatively
bacteriological study
cytokine measurement
patient
group
symptomatic
clinical evidence
anastomotic leakage
clinical evidence
AL
study
variable
hospital stay
wound infection
operative time
blood loss
height of anastomosis
intraperitoneal cytokine
microbiological study
peritoneal fluid
AL
patient
diagnose
postoperatively
median day
intraperitoneal
bacterial colonization
cytokine level
patient
clinical evidence
AL
wound infection
anastomotic leakage group
hospital stay
patient
anastomotic leakage
AL
group
operative time
blood loss
blood transfusion
height of the anastomosis
peritoneal
cytokine level
intraperitoneal
bacterial colonization
diagnostic tool
decision make
surgeon
early detection
anastomotic leak
colorectal

In [30]:
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [ ]:
for filename in tqdm(os.listdir(DATA_CLEAN_PATH)):
    with open(os.path.join(DATA_CLEAN_PATH, filename), "r") as f:
        text = f.read()
    doc = nlp_scispacy(text)
    for abrv in doc._.abbreviations:
        print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

  0%|          | 0/702 [00:00<?, ?it/s]

In [ ]:
print(doc._.abbreviations)

In [21]:
doc = nlp_scispacy("This is a dummy abbreviation test, also known as DAT")

/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [22]:
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [50]:
filename = "19912285_RE.csv"

with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
    relations_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

with open(os.path.join(ENTITIES_PATH, filename.split("_")[0]+".csv"), "r") as f:
    entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

In [51]:
typed_relations_df = relations_df.join(entities_df["Type"], on=["First"], rsuffix="_first")
typed_relations_df = typed_relations_df.join(entities_df["Type"], on=["End"], rsuffix="_end")

In [53]:
relations_df = relations_df.iloc[typed_relations_df.loc[typed_relations_df["Type"] == "ENTITY"].loc[typed_relations_df["Type_end"] == "ENTITY"].index].reset_index(drop=True)

In [57]:
typed_relations_df = relations_df.join(entities_df["Type"], on=["First"], rsuffix="_first")
typed_relations_df = typed_relations_df.join(entities_df["Type"], on=["End"], rsuffix="_end")
relations_df = relations_df.iloc[typed_relations_df.loc[typed_relations_df["Type"] == "ENTITY"].loc[typed_relations_df["Type_end"] == "ENTITY"].index].reset_index(drop=True)

In [306]:
total_sum = 0
num_entries = 0
max_dist = 0
cnt_files = 0
shitty_quantile = 0
p = 0.8

for filename in os.listdir(RELATIONS_PATH):
    
    cnt_files += 1
    
    with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
        relations_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)
    
    max_dist = max(max_dist, relations_df["Distance"].max())
    total_sum += sum(relations_df["Distance"])
    num_entries += len(relations_df)
    shitty_quantile += relations_df["Distance"].quantile(p)

In [304]:
print(total_sum/num_entries)
print(max_dist)

49.774383215743555
101


In [307]:
print(shitty_quantile/cnt_files)

79.17261055634809


In [310]:
sorted(("jajajja", "ahis is sparta"))

['ahis is sparta', 'jajajja']